In [70]:
import pandas as pd
import sqlite3

### Exercice 1 - Enrichissement et nettoyage des données
---

**Objectif : Nettoyer et enrichir un jeu de données avant chargement.**

Exemple :
Lire le fichier CSV 'clients.csv' (nom, ville, email, âge, dépenses, etc.).Certaines lignes contiennent des valeurs manquantes ou erronées.

Tâches :
- Nettoyer les valeurs manquantes (fillna, dropna).
- Supprimer les doublons (drop_duplicates).
- Calculer une colonne dépense_moyenne = total/nb_achats.
- Normaliser la casse (majuscules/minuscules).
- Charger dans SQLite (clients.db, table clients_propres).

Notions :
- Nettoyage de données (isna, dropna, replace)
- Gestion de la qualité des données
- Types de colonnes et conversions (astype)

In [71]:
# Chargement des données depuis le fichier CSV
df_clients = pd.read_csv('clientsTP2.csv')
print(df_clients.shape)
print(df_clients.columns)
df_clients.head()

(30, 7)
Index(['client_id', 'nom', 'ville', 'email', 'age', 'total_depenses',
       'nb_achats'],
      dtype='object')


,client_id,nom,ville,email,age,total_depenses,nb_achats
0,1,Client_1,NaN,client1@mail.com,40.0,1130.87,19
1,2,Client_2,Paris,client2@mail.com,25.0,731.74,12
2,3,Client_3,Toulouse,client3@mail.com,30.0,1645.60,3
3,4,Client_4,Toulouse,client4@mail.com,30.0,1882.18,3
4,5,Client_5,Toulouse,NaN,30.0,2893.96,4


In [72]:
df_clients.isnull().sum()

client_id         0
nom               0
ville             4
email             6
age               5
total_depenses    0
nb_achats         0
dtype: int64

In [73]:
# Gestion des lignes avec des valeurs manquantes
df_clients.fillna(value={
    'age': df_clients['age'].median(), # Remplacer les âges manquants par la médiane
    'ville': 'Inconnu', # Remplacer les villes manquantes par 'Inconnu'
    'email': df_clients['nom'].str.replace('_', '').str.lower() + '@mail.com' # Générer des emails basés sur le nom
}, inplace=True)
print(df_clients.shape)
df_clients.head()

(30, 7)


,client_id,nom,ville,email,age,total_depenses,nb_achats
0,1,Client_1,Inconnu,client1@mail.com,40.0,1130.87,19
1,2,Client_2,Paris,client2@mail.com,25.0,731.74,12
2,3,Client_3,Toulouse,client3@mail.com,30.0,1645.60,3
3,4,Client_4,Toulouse,client4@mail.com,30.0,1882.18,3
4,5,Client_5,Toulouse,client5@mail.com,30.0,2893.96,4


In [74]:
# Suppression des doublons
df_clients.drop_duplicates(inplace=True)
print(df_clients.shape)

(30, 7)


In [75]:
# Nouvelle colonne 'depense_moyenne' calculée comme 'total_depense' divisé par 'nb_achats'
df_clients['depense_moyenne'] = df_clients['total_depenses'] / df_clients['nb_achats']
df_clients.head()

,client_id,nom,ville,email,age,total_depenses,nb_achats,depense_moyenne
0,1,Client_1,Inconnu,client1@mail.com,40.0,1130.87,19,59.519474
1,2,Client_2,Paris,client2@mail.com,25.0,731.74,12,60.978333
2,3,Client_3,Toulouse,client3@mail.com,30.0,1645.60,3,548.533333
3,4,Client_4,Toulouse,client4@mail.com,30.0,1882.18,3,627.393333
4,5,Client_5,Toulouse,client5@mail.com,30.0,2893.96,4,723.490000


In [76]:
# Normaliser la casse (majuscules/minuscules). -> ????


In [77]:
# Chargement dans une base de données SQLite
conn = sqlite3.connect('TP2.db')
df_clients.to_sql('clients', conn, if_exists='replace', index=False)
conn.close()

### Exercice 2 - Fusion de plusieurs sources de données
---

**Objectif : Combiner plusieurs CSV pour créer une base analytique.**

Exemple :
- ventes.csv → ventes par produit
- produits.csv → info produits (catégorie, prix, fournisseur)
- clients.csv → info client

Tâches :
- Lire les 3 fichiers CSV.
- Fusionner (merge) pour créer une table complète : ventes + clients + produits.
- Calculer chiffre d’affaires total par client
- Calculer chiffre d’affaires par catégorie de produit.
- Charger les résultats dans SQLite (analyse.db).*

Notions :
- merge, groupby, agg
- Agrégations et jointures
- Création de tables analytiques

In [78]:
df_ventes = pd.read_csv('ventesTP2.csv')
print(df_ventes.shape)
print(df_ventes.columns)
df_ventes.head()

(100, 5)
Index(['vente_id', 'client_id', 'produit_id', 'Quantite', 'Prix_unitaire'], dtype='object')


,vente_id,client_id,produit_id,Quantite,Prix_unitaire
0,1,22,31,6,65.46
1,2,8,19,4,167.15
2,3,8,24,7,162.37
3,4,12,2,5,129.80
4,5,23,7,8,76.78


In [79]:
df_produits = pd.read_csv('produitsTP2.csv')
print(df_produits.shape)
print(df_produits.columns)
df_produits.head()

(40, 5)
Index(['produit_id', 'Produit', 'categorie', 'fournisseur', 'Prix_unitaire'], dtype='object')


,produit_id,Produit,categorie,fournisseur,Prix_unitaire
0,1,Produit_1,Accessoires,FournC,85.22
1,2,Produit_2,Accessoires,FournB,32.29
2,3,Produit_3,Bureau,FournB,11.48
3,4,Produit_4,Informatique,FournA,196.60
4,5,Produit_5,Informatique,FournA,77.79


In [80]:
print(df_clients.shape)
print(df_clients.columns)
df_clients.head()

(30, 8)
Index(['client_id', 'nom', 'ville', 'email', 'age', 'total_depenses',
       'nb_achats', 'depense_moyenne'],
      dtype='object')


,client_id,nom,ville,email,age,total_depenses,nb_achats,depense_moyenne
0,1,Client_1,Inconnu,client1@mail.com,40.0,1130.87,19,59.519474
1,2,Client_2,Paris,client2@mail.com,25.0,731.74,12,60.978333
2,3,Client_3,Toulouse,client3@mail.com,30.0,1645.60,3,548.533333
3,4,Client_4,Toulouse,client4@mail.com,30.0,1882.18,3,627.393333
4,5,Client_5,Toulouse,client5@mail.com,30.0,2893.96,4,723.490000


In [81]:
# Fusion des DataFrames
df_merged = pd.merge(df_ventes, df_clients, on='client_id', how='left')
df_merged = pd.merge(df_merged, df_produits, on='produit_id', how='left')
df_merged.sort_values(by='client_id', inplace=True)
print(df_merged.shape)
print(df_merged.columns)
df_merged.head()

(100, 16)
Index(['vente_id', 'client_id', 'produit_id', 'Quantite', 'Prix_unitaire_x',
       'nom', 'ville', 'email', 'age', 'total_depenses', 'nb_achats',
       'depense_moyenne', 'Produit', 'categorie', 'fournisseur',
       'Prix_unitaire_y'],
      dtype='object')


,vente_id,client_id,produit_id,Quantite,Prix_unitaire_x,nom,ville,email,age,total_depenses,nb_achats,depense_moyenne,Produit,categorie,fournisseur,Prix_unitaire_y
60,61,1,28,7,92.27,Client_1,Inconnu,client1@mail.com,40.0,1130.87,19,59.519474,Produit_28,Accessoires,FournA,198.52
55,56,1,37,8,30.71,Client_1,Inconnu,client1@mail.com,40.0,1130.87,19,59.519474,Produit_37,Informatique,FournB,47.66
44,45,1,23,4,62.77,Client_1,Inconnu,client1@mail.com,40.0,1130.87,19,59.519474,Produit_23,Informatique,FournC,121.35
77,78,1,1,3,143.18,Client_1,Inconnu,client1@mail.com,40.0,1130.87,19,59.519474,Produit_1,Accessoires,FournC,85.22
28,29,1,3,2,28.61,Client_1,Inconnu,client1@mail.com,40.0,1130.87,19,59.519474,Produit_3,Bureau,FournB,11.48


In [ ]:
# Calcule du chiffre d'affaire par vente
df_merged["chiffre_affaire"] = df_merged["Quantite"] * df_merged["Prix_unitaire_x"]

In [ ]:
# Calcule du chiffre d'affaire total par client
chiffre_affaire_par_client = (
    df_merged.groupby("client_id", as_index=False)
      .agg(chiffre_affaire_par_client=("chiffre_affaire", "sum"))
)
chiffre_affaire_par_client.head()

,client_id,chiffre_affaire_par_client
0,1,1711.93
1,3,203.46
2,4,863.40
3,5,3359.68
4,6,1603.53


In [87]:
# Calcule du chiffre d'affaire total par categorie de produit
chiffre_affaire_par_categorie = (
    df_merged.groupby("categorie", as_index=False)
      .agg(chiffre_affaire_par_categorie=("chiffre_affaire", "sum"))
)
chiffre_affaire_par_categorie.head()

,categorie,chiffre_affaire_par_categorie
0,Accessoires,13279.57
1,Bureau,16058.50
2,Informatique,24590.68


In [89]:
# Chargement dans une base de données SQLite
conn = sqlite3.connect('analyse.db')
chiffre_affaire_par_client.to_sql('chiffre_affaire_par_client', conn, if_exists='replace', index=False)
chiffre_affaire_par_categorie.to_sql('chiffre_affaire_par_categorie', conn, if_exists='replace', index=False)
conn.close()

### Exercice 3 - ELT (Transformation dans la base SQLite)
---

**Objectif : Réaliser les transformations directement dans SQL.**

Étapes :
- Charger les données brutes dans SQLite (to_sql).
- Écrire des requêtes SQL :
```sql
CREATE TABLE ventes_filtrees AS
SELECT Produit, SUM(Quantite * Prix_unitaire) AS Revenu
FROM ventes_brutes
```

GROUP BY Produit;
1. Lire la table ventes_filtrees avec Pandas.


Notions :
- Requêtes SQL avec conn.execute()
- Transformation “in-database” (ELT)
- Lecture SQL → Pandas (read_sql())